In [80]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from shapely.geometry import Polygon
import geopandas as gpd

from sentinelhub import FisRequest, BBox, Geometry, CRS, WcsRequest, CustomUrlParam, DataCollection, HistogramType
from sentinelhub.time_utils import parse_time

In [81]:
from sentinelhub import SHConfig


INSTANCE_ID = '3704e2d2-43dc-4f34-a563-13ec19dff111'  # In case you put instance ID into configuration file you can leave this unchanged

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

In [82]:
single_region = gpd.read_file("Metropolitan Manila.geojson")
single_region_geometry = Geometry(single_region.geometry.values[0], crs=CRS.WGS84)
single_region_geometry

Geometry(POLYGON ((121.072083 14.374083, 121.012344 14.364105, 120.929947 14.469853, 120.886002 14.646652, 121.012344 14.799409, 121.128387 14.772186, 121.109161 14.569569, 121.098862 14.490464, 121.072083 14.374083)), crs=CRS('4326'))

In [99]:
single_region_geometry_n = FisRequest(
    data_collection=DataCollection.SENTINEL5P,
    layer='NO2_SENTINEL5P',
    geometry_list=[single_region_geometry],
    #time=('2020-06-20', '2020-12-31'),
    #time=('2021-01-01', '2021-06-30'),
    time=('2021-07-01', '2021-12-31'),
    resolution='1000m',
    config=config
)

single_region_geometry_stats_n = single_region_geometry_n.get_data()

C:\Users\Administrator\anaconda3\lib\site-packages\sentinelhub\api\fis.py:97: SHDeprecationWarning: Fis service is being deprecated in favour of SentinelHubStatistical. Although no immediate action is needed as FIS is still supported, consider switching to Statistical API because it provides additional functionalities.
  warnings.warn(message, category=SHDeprecationWarning)


In [100]:
# request will return json output
single_region_geometry_stats_n

[{'C0': [{'date': '2021-12-31',
    'basicStats': {'min': 0.0,
     'max': 1.0,
     'mean': 0.2633854089716265,
     'stDev': 0.35159850897587763}},
   {'date': '2021-12-30',
    'basicStats': {'min': 0.0,
     'max': 1.0,
     'mean': 0.188345294714074,
     'stDev': 0.3115598725576618}},
   {'date': '2021-12-29',
    'basicStats': {'min': 0.0,
     'max': 1.0,
     'mean': 0.3517304052443198,
     'stDev': 0.429503737823417}},
   {'date': '2021-12-28',
    'basicStats': {'min': 'NaN', 'max': 'NaN', 'mean': 'NaN', 'stDev': 'NaN'}},
   {'date': '2021-12-27',
    'basicStats': {'min': 'NaN', 'max': 'NaN', 'mean': 'NaN', 'stDev': 'NaN'}},
   {'date': '2021-12-26',
    'basicStats': {'min': 'NaN', 'max': 'NaN', 'mean': 'NaN', 'stDev': 'NaN'}},
   {'date': '2021-12-25',
    'basicStats': {'min': 0.0, 'max': 0.0, 'mean': 0.0, 'stDev': 0.0}},
   {'date': '2021-12-24',
    'basicStats': {'min': 0.0,
     'max': 1.0,
     'mean': 0.2064056307374442,
     'stDev': 0.3082235283592262}},
   {'da

In [101]:
def fis_data_to_dataframe(fis_data):
    """Creates a DataFrame from list of FIS responses"""
    COLUMNS = ["channel", "date", "min", "max", "mean", "stDev"]
    data = []

    for fis_response in single_region_geometry_stats_n:
        for channel, channel_stats in fis_response.items():
            for stat in channel_stats:
                row = [int(channel[1:]), parse_time(stat["date"], force_datetime=True)]

                for column in COLUMNS[2:]:
                    row.append(stat["basicStats"][column])

                data.append(row)

    return pd.DataFrame(data, columns=COLUMNS).sort_values(["channel", "date"])


df = fis_data_to_dataframe(single_region_geometry_stats_n)

df.head()

,channel,date,min,max,mean,stDev
183,0,2021-07-01,0.0,1.0,0.459852,0.305251
182,0,2021-07-02,0.0,1.0,0.766336,0.284699
181,0,2021-07-03,NaN,NaN,NaN,NaN
180,0,2021-07-04,NaN,NaN,NaN,NaN
179,0,2021-07-05,0.0,1.0,0.507126,0.430254


In [102]:
def clean_df (json_stats):
    raw_df_stats = fis_data_to_dataframe(json_stats).reset_index(drop=True)
    index_df = raw_df_stats[raw_df_stats.channel ==0]
    cloud_df = raw_df_stats[raw_df_stats.channel ==1]
    cloud_df = cloud_df.drop(['channel'], axis=1).add_suffix('_clm')
    merged_df = pd.merge(index_df, cloud_df, left_on ='date', right_on = 'date_clm', how='outer')
    clean_merged_df = merged_df.drop(['channel', 'date_clm'], axis=1)
    return clean_merged_df

In [103]:
def clean_df (json_stats):
    raw_df_stats = fis_data_to_dataframe(json_stats).reset_index(drop=True)
    index_df = raw_df_stats[raw_df_stats.channel ==0]
    cloud_df = raw_df_stats[raw_df_stats.channel ==1]
    return cloud_df

In [104]:
clean_df

<function __main__.clean_df(json_stats)>

In [105]:
# Data Frame Cleaning
single_region_stats_clean = clean_df(single_region_geometry_stats_n)
single_region_stats_clean

,channel,date,min,max,mean,stDev
184,1,2021-07-01,0.0,1.0,0.628779,0.435724
185,1,2021-07-02,0.0,1.0,0.459928,0.409427
186,1,2021-07-03,NaN,NaN,NaN,NaN
187,1,2021-07-04,NaN,NaN,NaN,NaN
188,1,2021-07-05,0.0,1.0,0.55298,0.429551
...,...,...,...,...,...,...
363,1,2021-12-27,NaN,NaN,NaN,NaN
364,1,2021-12-28,NaN,NaN,NaN,NaN
365,1,2021-12-29,0.0,1.0,0.801555,0.266912
366,1,2021-12-30,0.0,1.0,0.709801,0.34124


In [106]:
# Export to CSV
single_region_stats_clean.to_csv('regional_no2_csv_2022/NCR3.csv', index=False)